In [17]:
def reuters_scrap(keyword):
    import webbrowser
    import time
    import requests
    from bs4 import BeautifulSoup
    import os
    import re
    from selenium import webdriver
    chrome_path=r"C:\Users\ADMIN\Desktop\CIMB Work\chromedriver.exe"
    from datetime import datetime, timedelta
    todaysdate=datetime.now().strftime("%Y-%m-%d")
    last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
    import warnings
    warnings.filterwarnings('ignore')
    url_base=[]
    headline=[]
    headline_url=[]
    cat=[]

    for key in keyword:
        base_url="https://www.reuters.com/site-search/?query={}&date=past_24_hours&offset=0".format(key)
        driver=webdriver.Chrome(chrome_path)
        driver.get(base_url)
        time.sleep(3)
        real_soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()
        ab=real_soup.find_all("li", {"class":"search-results__item___178TyG"})
        if(ab!=None):
            for i in ab:
                link=i.find("a",href=True)
                link1=link['href'].strip()
                link2="https://www.reuters.com"+link1
                headline_url.append(link2)
                head_title=link.get_text().strip()
                headline.append(head_title)
                url_base.append(base_url)
                cat.append(key)
        else:
            headline_url.append("No url found")
            headline.append("No headline found")
            url_base.append(base_url)
            cat.append(key)
            
    import pandas as pd
    df = pd.DataFrame({'keyword':cat,'main_url':url_base, 'Headline':headline,'headline_url': headline_url })
    df.drop_duplicates(inplace=True)        
    import nltk
    nltk.download('punkt')
    import webbrowser
    import re
    import time
    from newspaper import Article
    from newspaper import Config
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'
    config = Config()
    config.browser_user_agent = user_agent
    title=[]
    summary=[]
    article=[]
    key=[]
    date_time=[]
    url_1=[]
    for url in df.headline_url:
        if url!='No url found':
            time.sleep(5)
            url_1.append(url)
            toi_article = Article(url, language="en",config=config) # en for English
            toi_article.download()
            toi_article.parse()
            toi_article.nlp()
            title.append(toi_article.title)
            article.append(toi_article.text)
            summary.append(toi_article.summary)
            key.append(toi_article.keywords)
            date_time.append(toi_article.publish_date)

        else:
            url_1.append(url)
            title.append("")
            article.append("")
            summary.append("")
            key.append("")
            date_time.append("")
            
    df["article"]=article
    df["date_time"]=date_time
    df["title"]=title
    df["summary"]=summary
    df["key"]=key
    df["url"]=url_1
    df["source"]="reuters"  
    df2=df[['keyword','source','url','date_time','title','summary','article']].copy()

#     from fuzzywuzzy import fuzz
#     df2['Match'] = df2.apply(lambda x: fuzz.partial_ratio(x['keyword'], x['article']), axis=1)
#     df2['Match_1'] = df2.apply(lambda x: fuzz.token_set_ratio(x['keyword'], x['article']), axis=1)
    
    return df2

In [18]:
keyword=['RP TRADING','CIMB bank','RHB bank','maybank']
df_reuters=reuters_scrap(keyword)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
df_reuters

,keyword,source,url,date_time,title,summary,article


In [14]:
import time
from datetime import datetime, timedelta
todaysdate=datetime.now().strftime("%Y-%m-%d")
last_date=(datetime.now() + timedelta(days=-6)).strftime("%Y-%m-%d")
df_reuters['date_time']

0    2022-03-01
1    2022-03-01
2    2022-03-01
3    2022-03-01
4    2022-03-01
5    2022-02-28
6    2022-03-01
7    2022-02-26
8    2022-03-01
9    2022-03-01
10   2022-03-01
11   2022-03-01
12   2022-03-01
13   2022-02-28
14   2022-03-01
15   2022-03-01
16   2022-03-01
17   2022-03-01
18   2022-03-01
19   2022-03-01
20   2022-03-01
21   2022-03-01
22   2022-03-01
23   2022-02-28
24   2022-03-01
25   2022-03-01
26   2022-03-01
27   2022-03-01
28   2022-03-01
29   2022-03-01
Name: date_time, dtype: datetime64[ns]

In [12]:
import numpy as np
df_reuters['trigger']=np.where(((df_reuters['date_time']>=last_date)
                                 &(df_reuters['date_time']<=todaysdate))
                                & (df_reuters['Match_1']>90),'Y','N')

In [13]:
df_reuters['trigger']

0     N
1     N
2     N
3     N
4     N
5     N
6     N
7     N
8     N
9     N
10    N
11    N
12    N
13    N
14    N
15    N
16    N
17    N
18    N
19    N
20    N
21    N
22    N
23    N
24    N
25    N
26    N
27    N
28    N
29    N
Name: trigger, dtype: object